# Encontre o Valor Mais Próximo na BST

Escreva uma função que recebe uma Árvore de Busca Binária (BST) e um valor inteiro alvo e retorna o valor mais próximo a esse valor alvo contido na BST.

Você pode assumir que haverá apenas um valor mais próximo.

Cada nó da BST tem um valor inteiro, um nó filho esquerdo e um nó filho direito. Um nó é considerado um nó válido da BST se e somente se satisfizer a propriedade da BST:

- Seu valor é estritamente maior que os valores de todos os nós à sua esquerda;
- Seu valor é menor ou igual aos valores de todos os nós à sua direita;
- Seus nós filhos são ou nós válidos da BST ou None/nulo.

## Exemplo de entrada

In [96]:
'''
tree =   10
       /     \
      5       15
    /   \   /   \
   2    5  13    22
  /          \
 1           14
'''
target = 12

## Exemplo de saída

In [2]:
13

13

**Dica 1**

Tente percorrer a árvore BST nó por nó, mantendo o controle do nó com o valor mais próximo ao valor alvo. Calcular o valor absoluto da diferença entre o valor de um nó e o valor alvo deve permitir que você verifique se aquele nó está mais próximo do que o nó mais próximo atual.

**Dica 2**

Faça uso da propriedade da BST para determinar de qual lado de qualquer nó dado estão os valores próximos ao valor alvo e, portanto, que vale a pena explorar.

**Dica 3**

Quais são as vantagens e desvantagens de resolver este problema de forma iterativa em vez de recursiva?

**Complexidade Ótima de Tempo e Espaço**

- Média: O(log(N)) tempo | O(1) espaço.
 > onde n é o número de nós na BST.
- Pior caso: O(n) tempo | O(1) espaço
 > onde n é o número de nós na BST.

## Definição de classes

In [98]:
!pip install pytest pytest-sugar

In [99]:
import pytest
import time
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go


# Definindo a classe da árvore binária de busca (BST)
class BST:
    class Node:
        def __init__(self, value):
            self.value = value
            self.left = None
            self.right = None

    def __init__(self):
        self.root = None

    def add(self, value):
        if not self.root:
            self.root = self.Node(value)
        else:
            self._add(self.root, value)

    def _add(self, node, value):
        if value < node.value:
            if node.left is None:
                node.left = self.Node(value)
            else:
                self._add(node.left, value)
        else:
            if node.right is None:
                node.right = self.Node(value)
            else:
                self._add(node.right, value)

    def in_order(self, node=None):
        if node is None:
            node = self.root
        elements = []
        if node.left:
            elements.extend(self.in_order(node.left))
        elements.append(node.value)
        if node.right:
            elements.extend(self.in_order(node.right))
        return elements

# Função para encontrar o valor mais próximo ao target na árvore binária de busca
def find_closest_value(bst, target):
    return _find_closest_value(bst.root, target, float('inf'), None)

def _find_closest_value(node, target, closest_diff, closest_value):
    if node is None:
        return closest_value
    
    current_diff = abs(node.value - target)
    
    if current_diff < closest_diff:
        closest_diff = current_diff
        closest_value = node.value
    
    if target < node.value:
        return _find_closest_value(node.left, target, closest_diff, closest_value)
    elif target > node.value:
        return _find_closest_value(node.right, target, closest_diff, closest_value)
    else:
        return closest_value


# Testes da função find_closest_value
def test_find_closest_value():
    bst = BST()
    for value in [15, 5, 20, 17, 22, 2, 5, 1, 3]:
        bst.add(value)

    # Testes de valores próximos
    assert find_closest_value(bst, 18) == 17  # Mais próximo de 18
    assert find_closest_value(bst, 10) == 5   # Mais próximo de 10
    assert find_closest_value(bst, 1) == 1    # O valor exato (1)

# Função para medir o tempo de execução
def measure_execution_time(array_sizes, num_trials, target):
    times = []
    for size in array_sizes:
        total_time = 0
        trial_times = []
        for _ in range(num_trials):
            # Gerar dados aleatórios e adicionar ao BST
            data = np.random.randint(1, 1000, size=size)
            bst = BST()
            for value in data:
                bst.add(value)
            
            # Medição do tempo para encontrar o valor mais próximo de 'target'
            start_time = time.time()
            find_closest_value(bst, target)
            end_time = time.time()
            
            trial_time = end_time - start_time
            trial_times.append(trial_time)
            total_time += trial_time

        # Calcular o tempo médio e o intervalo de confiança
        mean_time = total_time / num_trials
        conf_interval = np.std(trial_times) / np.sqrt(num_trials)  # Intervalo de confiança
        times.append((mean_time, conf_interval))

    return times

# Função para plotar os tempos de execução
def plot_execution_times(array_sizes, times):
    mean_times = [t[0] for t in times]
    conf_intervals = [t[1] for t in times]

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=array_sizes,
        y=mean_times,
        mode='lines+markers',
        name='Execution Time',
        error_y=dict(type='data', array=conf_intervals, visible=True)
    ))

    fig.update_layout(
        title="Execution Time vs Input Size",
        xaxis=dict(title="Input Size"),
        yaxis=dict(title="Execution Time (s)"),
        template="plotly_white"
    )
    fig.show()


In [100]:
%run -i binarysearchtree.py

In [101]:
def test_find_kth_largest_value():
    bst = BST()
    for value in [15, 5, 20, 17, 22, 2, 5, 1, 3]:
        bst.add(value)
    assert find_closest_value(bst, 3) == 17
    assert find_closest_value(bst, 1) == 22
    assert find_closest_value(bst, 5) == 5


@pytest.fixture(scope="session")
def test_performance():
    array_sizes = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
    times = measure_execution_time(array_sizes, num_trials=10, k=5)
    plot_execution_times(array_sizes, times)

In [102]:
import pytest
from binarysearchtree import *  # Importando as classes Node e BST

def findClosestValue(tree, target):
    """
    Finds the value in a binary search tree that is closest to the given target value.
    
    This function begins the search for the closest value from the root of the binary search tree.
    It works by recursively (or sequentially) exploring the tree, narrowing down the search based on the target value
    and the current node's value. The closest value is constantly updated throughout the search process.

    Parameters:
    tree (BinarySearchTree): The binary search tree object in which to find the closest value.
                             It is expected to have a 'root' attribute that points to the root node of the tree.
    target (int or float): The target value for which the closest value in the binary search tree is sought.

    Returns:
    int or float: The value in the binary search tree that is closest to the target value.
    """
    return findClosestValueInBstHelper(tree.root, target, tree.root.value)

def findClosestValueInBstHelper(node, target, closest):
    if node is None:
        return closest
    if abs(target - closest) > abs(target - node.value):
        closest = node.value
    if target < node.value:
        return findClosestValueInBstHelper(node.left_child, target, closest)  # Corrigido: 'left_child' em vez de 'left'
    elif target > node.value:
        return findClosestValueInBstHelper(node.right_child, target, closest)  # Corrigido: 'right_child' em vez de 'right'
    else:
        return closest


@pytest.fixture(scope="session")
def data():
    array = [[10, 5, 15, 13, 22, 14, 2, 5, 1],
             [100, 5, 502, 204, 55000, 1001, 4500, 203, 205, 207,
              206, 208, 2, 15, 5, 22, 57, 60, 1, 3, -51, 1, 1, 1, 1, 1, -403]
             ]
    return array

def test_1(data):
    bst = BST()
    for value in data[0]:
      bst.add(value)
    assert findClosestValue(bst, 12) == 13

def test_2(data):
    bst = BST()
    for value in data[1]:
      bst.add(value)
    assert findClosestValue(bst, 100) == 100

def test_3(data):
    bst = BST()
    for value in data[1]:
      bst.add(value)
    assert findClosestValue(bst, 208) == 208

def test_4(data):
    bst = BST()
    for value in data[1]:
      bst.add(value)
    assert findClosestValue(bst, 4500) == 4500

def test_5(data):
    bst = BST()
    for value in data[1]:
      bst.add(value)
    assert findClosestValue(bst, 4501) == 4500

def test_6(data):
    bst = BST()
    for value in data[1]:
      bst.add(value)
    assert findClosestValue(bst, -70) == -51

def test_7(data):
    bst = BST()
    for value in data[1]:
      bst.add(value)
    assert findClosestValue(bst, 2000) == 1001

def test_8(data):
    bst = BST()
    for value in data[1]:
      bst.add(value)
    assert findClosestValue(bst, 6) == 5

def test_9(data):
    bst = BST()
    for value in data[1]:
      bst.add(value)
    assert findClosestValue(bst, 30000) == 55000

def test_10(data):
    bst = BST()
    for value in data[1]:
      bst.add(value)
    assert findClosestValue(bst, -1) == 1

def test_11(data):
    bst = BST()
    for value in data[1]:
      bst.add(value)
    assert findClosestValue(bst, 29751) == 55000

def test_12(data):
    bst = BST()
    for value in data[1]:
      bst.add(value)
    assert findClosestValue(bst, 29749) == 4500


In [103]:
!pytest closestvalue.py -vv

============================= test session starts =============================
platform win32 -- Python 3.10.11, pytest-8.3.4, pluggy-1.5.0 -- C:\Users\polia\AppData\Local\Programs\Python\Python310\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\polia\OneDrive\Documentos\repos\aed2\U2T2
plugins: sugar-1.0.0
collecting ... collected 11 items

closestvalue.py::test_1 PASSED                                           [  9%]
closestvalue.py::test_2 PASSED                                           [ 18%]
closestvalue.py::test_3 PASSED                                           [ 27%]
closestvalue.py::test_4 PASSED                                           [ 36%]
closestvalue.py::test_5 PASSED                                           [ 45%]
closestvalue.py::test_6 PASSED                                           [ 54%]
closestvalue.py::test_7 PASSED                                           [ 63%]
closestvalue.py::test_8 PASSED                                           [ 72%]
closestval

In [104]:
print(f"Array size: {size}, Mean Time: {mean_time}, Confidence Interval: {conf_interval}")

NameError: name 'size' is not defined

In [ ]:
array_sizes = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
times = measure_execution_time(array_sizes, num_trials=10, target=250)
plot_execution_times(array_sizes, times)

AttributeError: 'Node' object has no attribute 'left'